# Machine Learning for Graphs - Tutorial B: The Graph Convolutional Network

Fill in your names and group number here:

**NAME STUDENT A :**

**NAME STUDENT B :**

**GROUP NUMBER :**


Implementing a machine learning experiment with graph data is an important skill that you will learn as part of this course. This hands-on tutorial will help you develop this skill, as well as help you familiarize yourself with many of the steps and techniques that you will likely need to use for your final project.

Representation learning is the task of learning sensible representations for your samples given some downstream task. On graphs, representation learning is commonly used to learn vector representations of the nodes. These node representations are often called *embeddings vectors* or just *embeddings*. Graph Neural Networks (GNN) are ideal for learning node embeddings, since the identity of a node is a function of its neighbourhood (up to depth *d*) and since GNNs learn internal node representations by applying an aggregation operator on exactly this neighbourhood. Different models with various choices of aggregation operator have been introduced over the past couple of years, with the *convolutional* and *attention* operators being the more popular choices.

For this tutorial, you are asked to implement the original *Graph Convolutional Network* (GCN) and to replicate some of the classification experiments from the [paper](https://arxiv.org/abs/1609.02907) that introduced it [1]. To help you on your way, we have already prepared this Python Notebook.

You are asked to team up with another student and to work together on this tutorial. Please register your team by creating a new group and by adding both members.

    [1] Kipf, T. N., & Welling, M. Semi-supervised Classification With Graph Convolutional Networks (2017).
---

## NumPy and PyTorch

In this course we will make use of the [NumPy](https://numpy.org) package for working with vector data, and the [PyTorch](https://pytorch.org) machine learning package. Both of these are probably already installed in your environment as part of the first tutorial (Numpy as a dependency of PyTorch) but if this is not the case then running the following cell will install these packages for you.

**Run the cell below to install the NumPy and PyTorch packages in your Python environment**

In [ ]:
%pip install numpy torch

**Run the cells below to import the necessay packages and to set a manual seed**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

seed = 42  # for reproducability
torch.manual_seed(seed)

## Data Preparation

In the previous tutorial we used the *RDFlib* package to import the dataset. That the dataset was encoded using an open standard made this possible. This is not always the case, however: it is very common to come accross graph datasets that use an arbitrary encoding. In the case of the *Cora* dataset loaded below, the graph has been stored in two parts: the first a set of integer-encoded edges $[i, j]$, with $i$ and $j$ the indices of the nodes, and the second as a set of *n*-hot encoded node representations. Being a citation graph, the edges convey who cites who, whereas the node vectors $e_i$ represent a sparse bag-of-words with vocabulary $\Omega$ for which holds that $e_i[j] = 1$ if word $\Omega[j]$ occurs in the document and $0$ otherwise.

To import the Cora dataset we first process the raw files using NumPy and cast the generated arrays to the correct datatypes. Next, we generate a node-to-integer map and reindex the edges to ensure that their node identifiers match those of the nodes.

**Run the following cells to import and process the data**

In [ ]:
path = './data/'

data = np.genfromtxt(path + "cora.content", dtype = str)
edges = np.genfromtxt(path + "cora.cites", dtype = int)

In [ ]:
# these have the same order
features = data[:, 1:-1].astype(int)
labels = data[:, -1]
nodes = data[:, 0].astype(int)

n2i = {n:i for i,n in enumerate(nodes)}
edges_reindexed = np.array([[n2i[source], n2i[target]] for source, target in edges])

num_nodes = len(nodes)
num_edges = len(edges)
num_features = len(features[0])

In [ ]:
# inspect the data
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Number of features: {num_features}\n")

for i in range(5):
    print(f"Node ID: {nodes[i]}")
    print(f"Node features: {features[i]}")
    print(f"Node label: {labels[i]}\n")
    
print(f"Edges: \n{edges_reindexed[:5]}")

## Task 1: Vectorizing the graph

Since graph neural networks aggregate the information from the neighbourhoods of nodes, they need to know which nodes are adjacent to which other nodes. Because the information from those neighbours must also be aggregated from *their* neighbourhoods, these models thus need a relatively large amount of information about the structure of a graph. This information comes in the form of an *adjacency matrix* $A$, such that $A[i,j] = 1$ if there exists a link between nodes $i$ and $j$, and $0$ otherwise.

Of course, the adjacency matrix only tells the model which nodes to aggregate. To also know *what* to aggregate, we need another matrix which uniquely identifies each node. This matrix is often called the *node feature matrix* $X$. If our nodes comes with one or more attributes, or *features*, then we can fill up this matrix with the corresponding values. This is commonly done with *multimodal learning*. More often, however, it is easier to just ignore the node features (if any), and to let $X$ equal the identity matrix $I$ such that $X[i,j] = 1$ iff $i = j$ and $0$ otherwise.

Finally, since the downstream task is *node classification*, we need a vector representation, the *target vector* $y$, for the class labels that are used to compute the loss and accuracy scores. Since we need to calculate the gradients during this step, we need a numerical encoding for the labels. 

### Task 1a: Creating a feature matrix

Write a procedure to generate a node feature matrix that maps each node to its respective feature vector. The result should be a *sparse* float tensor `X`, such that `X[i]` refers to the feature vector of node `i`. Since the Cora dataset comes with integer-encoded node features (the bag-of-words) there is no need to generate an indentity matrix. Remember that the whole set of features is stored in variable `features`.

In [ ]:
...
X = ...

Run the following code to check your feature matrix

In [ ]:
# Check your feature matrix
X.to_dense()[:10,:10]

### Task 1b: Create an adjacency matrix

Write a procedure to generate the adjacency matrix for the Cora graph. The result should be a *sparse* float tensor `A`, such that `A[i,j]` equals `1` if there exists an edge between nodes `i` and `j`, and `0` otherwise. Be aware that the GCN requires all nodes to have a reflexive edge (loops) which ensures that the nodes remember their previous state when updating.  

In [ ]:
...
A = ...

Run the following code to check your adjacency matrix

In [ ]:
# Check your adjacency matrix by using the sum as proxy
print(f"The number of connections, {int(A.sum())}, must equal the number of edges, {num_edges}," 
      f" plus the number of nodes, {num_nodes}")
A.to_dense()[:10,:10]

### Task 1c: Create the target vector

Write a procedure to generate the target vector with integer-encoded class labels. The result should be a `long` vector `y_true`, such that `y_true[i]` holds the target label of node `i`. Note that, with PyTorch, different loss functions require differently formatted target vectors.

In [ ]:
...

y_true = ...

num_labels = ...

Run the following code to check your target vector

In [ ]:
print(f'number of unique labels: {num_labels}\n')

print(f'y: {y_true[:10]}')

## Task 2: Partition the dataset

To properly perform our experiments we first need to partition our data into a _train_ and _test_ split. These splits are used to train and test our model, respectively, and must be disjoint to avoid information leakage. Ideally, we would als create a _validation_ split to use for model selection and/or hyperparameter optimization, but we dispense with that for now.

Create a procedure to create a train and test split with a ratio of 4 to 1. The result should be two vectors, `train_idx` and `test_idx`, that contain indices that point to the actual data (a _mask_) that are randomly drawn from the set of all indices.

In [ ]:
num_train = ...
num_test = ...

# use mask
...
train_idx = ...
test_idx = ...

Run the following code to check your partitions

In [ ]:
print(f"number of training samples: {num_train}")
print(f"number of testing samples: {num_test}")

print(f"\ntrain indices:\n{train_idx[:5]}")
print(f"\ntest indices:\n{test_idx[:5]}")

## The Graph Convolutional

The *Graph Convolutional Network* (GCN) is arguably the first major breakthrough in GNN development. Developed in 2017, the GCN introduces the idea of the *spectral graph convolution*, which, analogues to its visual counterpart, aggregates the information surrounding an object. In the case of *Convolutional Neural Networks* (CNN), these objects are pixels, whereas with the GCN these are nodes. This comparison becomes evident when you consider images as regular (grid-shaped) graphs with pixel as nodes.

The GCN is defined as a network with one or more *Graph Convolution* layers. Each of these layers applies the convolution operator to its input, and is defined as 

$$ H^{l+1} = \sigma(\tilde{D}^{- \frac{1}{2}} \tilde{A} \tilde{D}^{- \frac{1}{2}} H^l W^l) $$

where $\tilde{A}$ is the adjacency matrix with reflexive edges, $\tilde{D}$ the degree matrix derived from $\tilde{A}$, $H^l$ the internal node representations of layer $l$, $W^l$ the weight matrix of layer $l$, and $\sigma$ a nonlinearity like $ReLU$. Note that the initial node representation matrix $H^0 = X$.

In the experiments that we are reproducing the GCN is used for the task of node classification. For this purpose, the GCN is given two graph convolution layers, but with the nonlinearity of the last layer replaced by a softmax function:

$$ y = softmax(\hat{A}~\sigma(\hat{A} X W^0)~W^1) $$

with $\hat{A} = \tilde{D}^{- \frac{1}{2}} \tilde{A} \tilde{D}^{- \frac{1}{2}}$

### Task 3a: Implement the Graph Convolution

Implement the graph convolution layer as a subclass of PyTorch `nn.Module`. Concretely, you must implement the `__init__` and `forward` functions. Ensure that the computation supports sparse tensors, and that the input and output dimensions can be set on initialisation.

In [ ]:
class GraphConvolutionLayer(nn.Module):
    """
    A single Graph Convolution Layer
    """

    def __init__(self, ...):
        super().__init__()
        
        # your code here

    def forward(self, ...) -> torch.Tensor:
        # your code here

Run the following cell to initialize and test your implementation


In [ ]:
conv = GraphConvolutionLayer(...)
conv(...)

### Task 3b: Implement the Graph Convolutional Model

Implement the GCN as specified in the paper [1]. Concretely, implement a two-layer GCN with a ReLU activation function and dropout after the first layer, and with a softmax layer after the second.

In [ ]:
class GCN(nn.Module):
    def __init__(self, ...):
        super().__init__()

        # your code here

    def forward(self, ...) -> torch.Tensor:
        # your code here

Run the following cell to initialize and test your implementation

In [ ]:
model = GCN(...)

y_pred = model(...)
y_pred

## Training and testing

In normal circumstances the GCN updates its internal representation for all nodes in the graph after each pass. In other words, the GCN operates on the entire graph at once, rather than on just the training, test, or validation set. Since these sets are disjoint, it necessarily means that only part of the class labels are available each time. This is called *semi-supervised learning*. Because the model sees the entire graph each pass, it still outputs predictions for all the nodes. However, by just calculating the loss and accuracy on a specific split, we ensure that only the error on the nodes in that split is backpropagated.

### Task 4: Implementing evaluation metrics

Write a procedure to calculate the loss *and* a procedure to calculate the accuracy. Assume that we have a tensor with true labels, `y_true`, and a tensor with predicted labels, `y_pred`.

In [ ]:
loss_function = nn.NLLLoss()

def compute_accuracy(...) -> float:
    # your code here

def compute_loss(...) -> torch.Tensor:
    # your code here

Run the following cell to test your code:

In [ ]:
y_pred_labels = ...
print(f'Predicted labels: {y_pred_labels[:10]}')

acc = compute_accuracy(...)
print(f'Accuracy: {acc:.3f}')

loss = compute_loss(...)
print(f'Loss: {loss:.3f}')

### Task 5a: Implement the training loop

Write a procedure to train the model. Specifically, create a loop that passes the entire graph through the model every epoch, while computing the loss and accuracy on just the training set. Use the Adam optimizer and the negative log likelihood loss.

In [ ]:
# set hyperparameters
learning_rate = 0.01
num_epoch = 200

# set optimizer
optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate)

for epoch in range(1, num_epoch+1):
    print(f'Epoch {epoch:3d} - ', end='')

    # allow model parameters to be learned   
    model.train()         

    # your code here

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss = float(loss)  # release memory of computation graph

    print(f'loss: {loss:0.4f}\tacc: {acc:0.4f}')

### Task 5b: Implement the test procedure

Write a procedure to test the now-trained model. Ensure that the weights of your model are frozen during testing, and that the loss and accuracy scores are calculated on just the test set.

In [ ]:
# freeze model parameters for evaluation
model.eval()

# your code here

loss = float(loss)  # release memory of computation graph

print(f'test loss: {loss:0.4f}\ntest acc: {acc:0.4f}')